
Задание для личного проекта, который нужно будет сдавать для оценки:
# Написать чат бота, который будет говорить погоду в двух городах - Москва и СПб
# Бот должен уметь здороваться, прощаться.
# Бот должен понимать такие даты, как: сейчас, сегодня, завтра.
# Попрощаться - значит закончить беседу (после прощания, он не должен писать никаких сообщений пока вы не напишите)
# Бот должен как-то реагировать на непонятные высказывания
# Базу погоды можно составить самим - на два дня, на два города или использовать API (например, OpenWeatherMap (https://openweathermap.org/)).
# Бот должен работать и продолжать вас что-то спрашивать пока вы не закончите беседу.
# Бот должен запрашивать всю необходимую информацию для прогноза (если не хватает какого-то слота (город или дата), он должен его уточнить)

Один из сценариев скрипта:
1. Написать функцию распознания интенции (Например, Natasha)
2. Функция для того, чтобы здороваться.
3. Функция для погоды
4. Функция, чтобы прощаться.
5. Функция для ошибок.

То есть, из фич только погода.

Мануал как делать бота на питоне

(https://thecode.media/python-bot/)Если что-то не получается, все падает с ошибкой и т.д. - смело пишите сюда или в личку.

Deadline по проекту: сессионная неделя.
ДЗ можно просто отправить, я посмотрю, и если что-то не так, то доработать.
Проект нужно будет показывать. Можно показать, и если что-то не так, то доделать.

P.S. Каждый член команды должен будет ответить на вопросы по боту и рассказать как это работает.

P.P.S Не забывайте про домашнее задание - это почти половина оценки

In [20]:
import json
import urllib3
import spacy
from secret import *

from datetime import date
from datetime import datetime
import time
nlp = spacy.load('ru_core_news_md')

In [4]:
http = urllib3.PoolManager()

### geo

In [5]:
def place_to_geo(place, city, map_key = map_key):
    '''

    :param place: Name of place to search
    :param city: Name of city
    :param map_key: API KEY from https://developer.mapquest.com/
    :return: dict {'lat': '55.6280353', 'lon': '36.9952076310053'}
    '''
    geo = {}

    url = f'http://open.mapquestapi.com/nominatim/v1/search.php?key={map_key}&format=json&q={place}+{city}&addressdetails=1&limit=3'
    response = http.request("GET", url)
    data = json.loads(response.data)[0]

    geo['lat'] = data['lat']
    geo['lon'] = data['lon']
    return geo

In [6]:
geo_ll =  place_to_geo("Невский проспект", "Москва")
geo_ll

{'lat': '59.9316703', 'lon': '30.3588031'}

### weather

In [17]:
def location_to_weather(geo_ll, weather_key = weather_key):
    '''
    :param geo_ll: dict from place_to_geo
    :param weather_key: api key from https://openweathermap.org/api/one-call-api
    :return: dict with temp for 7 days
    '''
    weather = {}

    lat = geo_ll['lat']
    lon = geo_ll['lon']

    url = f'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=current,minutely,hourly,alerts&appid={weather_key}&units=metric'
    response = http.request("GET", url)
    data = json.loads(response.data)
    for day in data['daily']:
        weather[day['dt']]=  day['feels_like']
    return weather


In [18]:
weather = location_to_weather(geo_ll)
weather

{1639558800: {'day': -3.6, 'night': -3.01, 'eve': -2.9, 'morn': -2.98},
 1639645200: {'day': -1.49, 'night': -5.05, 'eve': -4.11, 'morn': -2.29},
 1639731600: {'day': -7.48, 'night': -9.68, 'eve': -6.92, 'morn': -7.24},
 1639818000: {'day': -6.39, 'night': -3.39, 'eve': -3.54, 'morn': -11.84},
 1639904400: {'day': -7.19, 'night': -14.67, 'eve': -11.01, 'morn': -7.31},
 1639990800: {'day': -18.36, 'night': -19.72, 'eve': -19.4, 'morn': -18.91},
 1640077200: {'day': -18.63, 'night': -21.74, 'eve': -20.61, 'morn': -20.97},
 1640163600: {'day': -20.76, 'night': -16.55, 'eve': -17.87, 'morn': -22.95}}

### part of speach

In [9]:
def text_to_location(text, nlp = nlp):
    '''
    :param text:
    :param nlp:
    :return:
    '''

    text_spacy = nlp(text)

    locations = []

    if len(text_spacy.ents) == 0:
        raise NameError("Нет локации")
    else:
        for ent in text_spacy.ents:
            if ent.label_ == 'LOC':
                locations.append(ent.lemma_)

    return(locations)

In [10]:
text_to_location("как дела в Саратове и томске")

['саратов', 'томск']

## date

In [11]:
def text_to_date(text, nlp = nlp):

    today = time.mktime(date.today().timetuple())
    tokens = []

    today_unix = time.mktime(date.today().timetuple())

    text_spacy = nlp(text)

    for token in text_spacy:
        tokens.append(token.lemma_)
    tokens
    if 'сегодня' in tokens:
        return 0
    elif 'завтра' in tokens:
        return 1
    elif 'сейчас' in tokens:
        return 0
    elif 'послезавтра' in tokens:
        return 1
    else:
        raise NameError('На какой день сказать погоду?')

In [12]:
text_to_date("Скажи погоду в Санкт-Петербурге на завтра")

1

## text to answer

In [13]:
# text  = "Скажи погоду Москве на завтра"
text = "сколько вешать в граммах в Москве"


In [42]:
def message_to_reply(text):
    try:
        location = text_to_location(text)
    except NameError:
        return ("А для какого места тебе нужна погода?")
    try:
        date_index = text_to_date(text)
    except NameError:
        return ("Cкажи, а на какой день делать прогноз?")
    try:
        geo_ll = place_to_geo("", location)
    except NameError:
        return ("Попробуй другую локацию")
    try:
        weather = location_to_weather(geo_ll)
    except NameError:
        return (f'Не смог найти погоду для {location}')
    try:
        weather_day = weather[list(weather.keys())[date_index]]
        day = date.fromtimestamp(list(weather.keys())[date_index])
        return f'Погода в городе {location[0]} {day} ' \
               f'днём: {weather_day["day"]} ;' \
               f'вечером {weather_day["eve"]}'
    except NameError:
        return (f' ошибка в последнем действии')


In [43]:
text  = "Скажи погоду Москве на завтра"
# text = "сколько вешать в граммах в Москве"

message_to_reply(text)

'Погода в городе москва 2021-12-16 днём: -4.18 ;вечером -3.33'